In [89]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import random
from time import sleep

In [90]:
h1 = {
'Cookie': 'bid=pWHRXljqlnw; _pk_id.100001.4cf6=1889b80fc0472f29.1700409010.; __utmc=30149280; __utmz=30149280.1700409010.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmc=223695111; __utmz=223695111.1700409010.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); _ga=GA1.1.5521471.1700409010; ll="118282"; __utma=30149280.5521471.1700409010.1700409010.1700411523.2; __utma=223695111.1586942768.1700409010.1700409010.1700411523.2; _vwo_uuid_v2=D6D254F941FF811FF3C5BC45942B6A3E4|40c2965b4880f3c13ff94c0a7cdc0e98; _ga_RXNMP372GL=GS1.1.1700413503.2.0.1700413503.60.0.0; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1700552142%2C%22https%3A%2F%2Fwww.google.com%2F%22%5D; _pk_ses.100001.4cf6=1; ap_v=0,6.0; dbcl2="276019366:j+LEQsNs20s"; ck=_dp9; frodotk_db="98c409fbb250fd78ca4c1df316f2f868"; push_noty_num=0; push_doumail_num=0',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
'Accept-Encoding': 'gzip, deflate',
'Host': 'movie.douban.com',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15',
'Accept-Language': 'zh-CN,zh-Hans;q=0.9',
'Connection': 'keep-alive'
}

In [108]:
ip_list = []

with open('proxy.txt', 'r') as f:
    for line in f.readlines():
        ip_list.append(line.strip())

#获取ip代理
def getip():
    proxy= ip_list[random.randint(0,len(ip_list)-1)]
    proxy = proxy.replace("\n","")
    proxies={
        'http':'http://'+str(proxy),
        #'https':'https://'+str(proxy),
    }
    return proxies

In [91]:
def trans_star(v_str):
    v_str = v_str[0]
    if v_str == 'allstar10':
        return '1'
    elif v_str == 'allstar20':
        return '2'
    elif v_str == 'allstar30':
        return '3'
    elif v_str == 'allstar40':
        return '4'
    elif v_str == 'allstar50':
        return '5'
    else:
        return 'NaN'

In [113]:
def get_short(movie_url, max_page, data_path, file_name):
    df = pd.DataFrame()
    path =data_path + file_name + '.csv'
    if os.path.exists(path):
        print("file exists")
        return 0
    for i in range(0, max_page):
        url = movie_url + 'comments?start=' + str(i*20) + '&limit=20&status=P&sort=new_score'
        res = requests.get(url, headers=h1, verify=False, proxies=getip())
        soup = BeautifulSoup(res.text, 'html.parser')
        user_names = soup.find_all('span', class_='comment-info')
        user_names = [i.find('a').text for i in user_names]
        stars = soup.find_all('span', class_='comment-info')
        stars = [i.find_all('span')[1]['class'] for i in stars]
        stars = [trans_star(i) for i in stars]
        comment_times = soup.find_all('span', class_='comment-time')
        comment_times = [i.text.strip() for i in comment_times]
        geo_ips = soup.find_all('span', class_='comment-location')
        geo_ips = [i.text for i in geo_ips]
        votes = soup.find_all('span', class_='votes vote-count')
        votes = [i.text for i in votes]
        shorts = soup.find_all('span', class_='short')
        shorts = [i.text for i in shorts]
        shorts = [i.replace(',', '，').replace(' ', '').replace('\n', '').replace('\r', '') for i in shorts]

        pd_data = {'user_name': user_names, 'star': stars, 'comment_time': comment_times, 'geo_ip': geo_ips, 'vote': votes, 'short': shorts}
        df = pd.concat([df, pd.DataFrame(pd_data)], axis=0)
        print('Page ' + str(i+1) + ' is done!')

    if len(df) < 10:
        print('No data!')
        return 1

    df.to_csv(path, index=False)
    return 0

In [114]:
#test
movie_url = 'https://movie.douban.com/subject/35267224/'
max_page = 2
data_path = './data/'
file_name = 'test'
get_short(movie_url, max_page, data_path, file_name)

c:\Users\shaoh\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Page 1 is done!


c:\Users\shaoh\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Page 2 is done!


0

In [133]:
# read movie list
movie_list = pd.read_csv('../analysis/MovieDouban250.csv')
# each line as a movie, convert to list of dict
movie_list = movie_list.to_dict('records')
print(movie_list)

# get short for each movie
for movie in movie_list:
    movie_url = movie['电影链接']
    max_page = 30
    data_path = './data/'
    file_name = movie['电影名称'].replace(' ', '').split('/')[0].replace('\xa0', '')
    ret = get_short(movie_url, max_page, data_path, file_name)
    if ret == 1:
        print("blocked!")
        break
    print(movie['电影名称'] + ' is done!')



[{'Unnamed: 0': 0, '电影名称': '肖申克的救赎\xa0/\xa0The Shawshank Redemption\xa0/\xa0月黑高飞(港)  /  刺激1995(台)', '电影链接': 'https://movie.douban.com/subject/1292052/', '电影评分': 9.7, '评分人数': 2944185, '导演': '导演: 弗兰克·德拉邦特 Frank Darabont', '主演': '主演: 蒂姆·罗宾斯 Tim Robbins /...', '上映年份': '1994', '国家': '美国', '类型': '犯罪 剧情'}, {'Unnamed: 0': 1, '电影名称': '霸王别姬\xa0/\xa0再见，我的妾  /  Farewell My Concubine', '电影链接': 'https://movie.douban.com/subject/1291546/', '电影评分': 9.6, '评分人数': 2175220, '导演': '导演: 陈凯歌 Kaige Chen', '主演': '主演: 张国荣 Leslie Cheung / 张丰毅 Fengyi Zha...', '上映年份': '1993', '国家': '中国大陆 中国香港', '类型': '剧情 爱情 同性'}, {'Unnamed: 0': 2, '电影名称': '阿甘正传\xa0/\xa0Forrest Gump\xa0/\xa0福雷斯特·冈普', '电影链接': 'https://movie.douban.com/subject/1292720/', '电影评分': 9.5, '评分人数': 2194424, '导演': '导演: 罗伯特·泽米吉斯 Robert Zemeckis', '主演': '主演: 汤姆·汉克斯 Tom Hanks / ...', '上映年份': '1994', '国家': '美国', '类型': '剧情 爱情'}, {'Unnamed: 0': 3, '电影名称': '泰坦尼克号\xa0/\xa0Titanic\xa0/\xa0铁达尼号(港 / 台)', '电影链接': 'https://movie.douban.com/subject/1292722/', '电影评分': 9.5,

c:\Users\shaoh\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\shaoh\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Page 1 is done!


c:\Users\shaoh\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\shaoh\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Page 2 is done!


...

c:\Users\shaoh\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\shaoh\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Page 29 is done!


c:\Users\shaoh\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Page 30 is done!
No data!
blocked!


c:\Users\shaoh\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
